# Choosing site basis sets for a cluster expansion

A variety of site basis sets are aviable in **smol**. In this notebook we show how to choose a specific basis when defining a `ClusterSubspace`.

In [1]:
from monty.serialization import loadfn
from smol.cofe import available_site_basis_sets, ClusterSubspace

### 0) List available site basis sets

You can print a list of the available site basis sets.

In [2]:
print(f"Available site basis sets:\n{available_site_basis_sets()}")

Available site basis sets:
('indicator', 'sinusoid', 'polynomial', 'chebyshev', 'legendre')


### 1) Choose a site basis for a `ClusterSubspace`

The names in the list can be used as key word arguments when constructing a `ClusterSubspace`. The default value is set to "indicator".

In [3]:
# load a prim cell
prim = loadfn("data/lmof_prim.json")
print(prim)

Full Formula (Li0.8 Mn1.35 O0.5 F0.5)
Reduced Formula: Li0.8Mn1.35O0.5F0.5
abc   :   2.969850   2.969850   2.969850
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (4)
  #  SP                                                a     b     c
---  ---------------------------------------------  ----  ----  ----
  0  Li+:0.300, Mn2+:0.300                          0.25  0.25  0.25
  1  Li+:0.200, Mn2+:0.250, Mn3+:0.250, Mn4+:0.250  0     0     0
  2  Li+:0.300, Mn2+:0.300                          0.75  0.75  0.75
  3  O2-:0.500, F-:0.500                            0.5   0.5   0.5


In [4]:
# default is indicator
subspace_indicator = ClusterSubspace.from_cutoffs(
    prim,
    cutoffs={2: 4.0, 3: 3.0},  # small cutoffs for illustration purpose
    supercell_size='O2-'
)

subspace_sine = ClusterSubspace.from_cutoffs(
    prim,
    cutoffs={2: 4.0, 3: 3.0},  # small cutoffs for illustration purpose
    basis="sinusoid",
    supercell_size='O2-'
)

print(subspace_indicator)

Basis/Orthogonal/Orthonormal : indicator/False/False
       Unit Cell Composition : Li+0.8 Mn2+0.85 Mn4+0.25 Mn3+0.25 O2-0.5 F-0.5
            Number of Orbits : 30
No. of Correlation Functions : 168
             Cluster Cutoffs : 2: 3.64, 3: 2.97
              External Terms : []
Orbit Summary
 ------------------------------------------------------------------------
 |  ID     Degree    Cluster Diameter    Multiplicity    No. Functions  |
 |   0       0             NA                 0                1        |
 |   1       1            0.0000              2                2        |
 |   2       1            0.0000              1                1        |
 |   3       1            0.0000              1                4        |
 |   4       2            1.8187              8                2        |
 |   5       2            1.8187              8                8        |
 |   6       2            2.1000              6                3        |
 |   7       2            2.1000      

### 1) Site basis set properties

Let look at the orthonormality and actual basis functions of the different basis sets.

In [5]:
# We can already see above that the default indicator basis is not orthogonal or orthonormal
# We can verify with the following
print(f"Chosen {subspace_indicator.basis_type} basis set is orthonormal: {subspace_indicator.basis_orthonormal}")
print(f"Chosen {subspace_indicator.basis_type} basis set is orthonormal: {subspace_indicator.basis_orthogonal}")

Chosen indicator basis set is orthonormal: False
Chosen indicator basis set is orthonormal: False


In [6]:
# The sinusoid basis is orthogonal (it is orthonormal only for binary systems)
print(f"Chosen {subspace_sine.basis_type} basis set is orthonormal: {subspace_sine.basis_orthonormal}")
print(f"Chosen {subspace_sine.basis_type} basis set is orthonormal: {subspace_sine.basis_orthogonal}")

Chosen sinusoid basis set is orthonormal: False
Chosen sinusoid basis set is orthonormal: True


In [7]:
# We can look at the actual basis set values from the basis arrays of singlet orbits
# In the arrays the each row is a basis function
print(subspace_indicator.orbits[0])

print(f"The basis functions in the {subspace_indicator.basis_type} basis are:\n")
print(f"{subspace_indicator.orbits[0].basis_arrays[0]}\n")

print(f"The basis functions in the {subspace_sine.basis_type} basis are:\n")
print(f"{subspace_sine.orbits[0].basis_arrays[0]}\n")

Orbit  1
    Multiplicity : 2   
   No. functions : 2   
No. symmetry ops : 24  
Function ids : [1, 2]
Base Cluster : 
  | Diameter : 0.0000
  |   Charge : 1.0
  | Centroid :    -0.000000     1.818654    -0.000000  ->     0.250000     0.250000     0.250000
  | Sites (1)
  | ------------------------------------------------------------------------------------------------------------------------
  | 0 vacA0+:0.333, Li+:0.333, Mn2+:0.333    -0.000000     1.818654    -0.000000   ->     0.250000     0.250000     0.250000
The basis functions in the indicator basis are:

[[1. 0. 0.]
 [0. 1. 0.]]

The basis functions in the sinusoid basis are:

[[-1.         0.5        0.5      ]
 [-0.        -0.8660254  0.8660254]]



In [8]:
# We can look at the actual basis set values from the basis arrays of singlet orbits
print(subspace_indicator.orbits[1],"\n")

print(f"The basis functions in the {subspace_indicator.basis_type} basis are:\n")
print(f"{subspace_indicator.orbits[1].basis_arrays[0]}\n")

print(f"The basis functions in the {subspace_sine.basis_type} basis are:\n")
print(f"{subspace_sine.orbits[1].basis_arrays[0]}\n")

Orbit  2
    Multiplicity : 1   
   No. functions : 1   
No. symmetry ops : 48  
Function ids : [3]
Base Cluster : 
  | Diameter : 0.0000
  |   Charge : -1.5
  | Centroid :     0.000000     3.637309     0.000000  ->     0.500000     0.500000     0.500000
  | Sites (1)
  | --------------------------------------------------------------------------------------------------------
  | 0 O2-:0.500, F-:0.500     0.000000     3.637309     0.000000   ->     0.500000     0.500000     0.500000 

The basis functions in the indicator basis are:

[[1. 0.]]

The basis functions in the sinusoid basis are:

[[-1.  1.]]



### 2) Changing basis types

Once you have created a subspace, you can change the basis set type without having to recreate it.

NOTE: Once a cluster expansion fit has been done with a specific basis set type, changing the basis set of the underlying `ClusterSubspace` does not update ECI accordingly!

In [9]:
subspace_cheby = subspace_indicator.copy()
subspace_cheby.change_site_bases("chebyshev")

In [10]:
print(f"Chosen {subspace_cheby.basis_type} basis set is orthonormal: {subspace_cheby.basis_orthonormal}")
print(f"Chosen {subspace_cheby.basis_type} basis set is orthonormal: {subspace_cheby.basis_orthogonal}")

Chosen chebyshev basis set is orthonormal: False
Chosen chebyshev basis set is orthonormal: False


#### But wait! Isn't the original Chebyshev basis from the [original 1984 paper](https://doi.org/10.1016/0378-4371(84)90096-7) orthonormal?

Yes, it is; but in the original publication the basis is explicity orthonormalized. In **smol** the Chebyshev basis is constructed from Chebyshev polynomials without any orthonormalization procedure.

To obtain the same basis as proposed by Sanchez et al. we just need to orthonormalize!

In [11]:
subspace_cheby.change_site_bases("chebyshev", orthonormal=True)

print(f"Chosen {subspace_cheby.basis_type} basis set is orthonormal: {subspace_cheby.basis_orthonormal}")
print(f"Chosen {subspace_cheby.basis_type} basis set is orthonormal: {subspace_cheby.basis_orthogonal}")

Chosen chebyshev basis set is orthonormal: True
Chosen chebyshev basis set is orthonormal: True


#### In fact we can orthonormalize any type of basis!

Although note that in some cases the basis will no longer be the same "type" of basis (notably for an indicator basis). Actually it is best to think of an "orthonormalized basis" as a category of equivalent basis sets, regardless of which basis we started from.

In [12]:
subspace_indicator_on = ClusterSubspace.from_cutoffs(
    prim,
    cutoffs={2: 4.0, 3: 3.0},  # small cutoffs for illustration purpose
    supercell_size='O2-',
    basis="indicator",
    orthonormal=True
)

In [13]:
print(f"Chosen {subspace_indicator_on.basis_type} basis set is orthonormal: {subspace_indicator_on.basis_orthonormal}")
print(f"Chosen {subspace_indicator_on.basis_type} basis set is orthonormal: {subspace_indicator_on.basis_orthogonal}")

Chosen indicator basis set is orthonormal: True
Chosen indicator basis set is orthonormal: True


In [14]:
# lets look at the actual basis set values for an indicator basis, and a normalized indicator basis

print(f"The basis functions in the {subspace_indicator.basis_type} basis are:\n")
print(f"{subspace_indicator.orbits[0].basis_arrays[0]}\n")

print(f"The basis functions in the orthonormal {subspace_indicator_on.basis_type} basis are:\n")
print(f"{subspace_indicator_on.orbits[0].basis_arrays[0]}\n")

The basis functions in the indicator basis are:

[[1. 0. 0.]
 [0. 1. 0.]]

The basis functions in the orthonormal indicator basis are:

[[-1.41421356  0.70710678  0.70710678]
 [-0.          1.22474487 -1.22474487]]



In [15]:
print(f"The basis functions in the {subspace_cheby.basis_type} basis are:\n")
print(f"{subspace_cheby.orbits[0].basis_arrays[0]}\n")

The basis functions in the chebyshev basis are:

[[-1.22474487 -0.          1.22474487]
 [-0.70710678  1.41421356 -0.70710678]]



Notice how the orthonormalized basis sets starting from indicator functions and chebyshev polynomials are remarkably similar. In fact, the are related by a rotation!